# Creating Master Dataframe for Modeling

In [1]:
import pandas as pd
import re

### Load Data

In [4]:
# Load each CSV file
menu_df = pd.read_csv('./data/menu_df.csv')
diabetic_food_df = pd.read_csv('./data/diabetic_friendly_foods.csv')
gi_df = pd.read_csv('./data/glycemic_index.csv')
patient_df = pd.read_csv('./data/patient.csv')
dexcom_cgm_df = pd.read_csv('./data/dexcom_cgm.csv')

# Inspect each DataFrame to determine the best way to combine them
print("Menu Data:", menu_df.shape)
print("Diabetic Food Data:", diabetic_food_df.shape)
print("Glycemic Index Data:", gi_df.shape)
print("Patient Data:", patient_df.shape)
print("Dexcom CGM Data:", dexcom_cgm_df.shape)

Menu Data: (700, 14)
Diabetic Food Data: (365, 11)
Glycemic Index Data: (54, 2)
Patient Data: (33568, 22)
Dexcom CGM Data: (17118, 2)


## Calculate Suitability Scores

In [8]:
# Set weights based on nutritional importance
carb_weight = 0.5
sugar_weight = 0.3
fiber_weight = 0.1
protein_weight = 0.1

In [10]:
# Set ideal nutritional values
carb_goal = 52.5
sugar_goal = 7.5
fiber_goal = 7.5
protein_goal = 20

In [34]:
# Create function to calculate suitability score
def suitability_score(carbohydrates, sugars, fiber, proteins):
    # Check if any required value is missing
    if pd.isna(carbohydrates) or pd.isna(sugars) or pd.isna(fiber) or pd.isna(proteins):
        # Return -1 as an indicator that the score couldn't be calculated
        return -1
        
    # Make sure score doesn't drop below 0
    carb_score = 1 - abs(carbohydrates - carb_goal) / carb_goal
    carb_score = max(0, carb_score)
    
    # Penalize high sugar content
    sugar_score = 1 - sugars / sugar_goal
    sugar_score = max(0, sugar_score)

    # Cap fiber_score at 1 if >= fiber_goal
    fiber_score = min(fiber / fiber_goal, 1)

    # Cap protein_score at 1 if >= protein_goal
    protein_score = min(proteins / protein_goal, 1)

    score = (carb_weight * carb_score) + (sugar_weight * sugar_score) + (fiber_weight * fiber_score) + (protein_weight * protein_score)

    return round(score, 3)

In [36]:
# Create suitability_score column
menu_df['score'] = menu_df.apply(lambda row: suitability_score(row['carbohydrates'], row['sugars'], row['fiber'], row['proteins']), axis=1)
menu_df.head()

,restaurant_name,food_name,serving_size,serving_unit,calories,carbohydrates,sugars,fats,saturated_fats,cholesterol,sodium,fiber,potassium,proteins,score
0,McDonald's Canada,Egg BLT McMuffin with Shredded Lettuce (McDona...,1.0,Serving,7.99,1.55,0.56,0.14,0.02,0.00,3.76,0.99,116.09,0.58,0.308
1,McDonald's,Cheeseburger,1.0,Serving,535.31,39.24,7.16,28.66,14.00,95.52,1176.09,2.39,443.77,30.27,0.519
2,McDonald's,Hamburger,1.0,Serving,540.14,40.27,NaN,26.56,10.52,122.04,791.00,NaN,569.52,34.28,-1.000
3,McDonald's,Honey,1.0,Serving,63.84,17.30,17.25,0.00,0.00,0.00,0.84,0.04,10.92,0.06,0.166
4,McDonald's,Hotcakes,1.0,Serving,90.80,11.32,NaN,3.88,0.85,23.60,175.60,NaN,52.80,2.56,-1.000


## Merging Food Data

### Inspect Columns and Food Names

In [153]:
print(diabetic_food_df.columns)
print(gi_df.columns)
print(menu_df.columns)
print(patient_df.columns)
print(dexcom_cgm_df.columns)

Index(['food_name', 'category', 'description', 'brand', 'food_category',
       'calories', 'carbohydrates', 'fiber', 'sugars', 'fats', 'proteins'],
      dtype='object')
Index(['food_name', 'glycemic_index'], dtype='object')
Index(['restaurant_name', 'food_name', 'serving_size', 'serving_unit',
       'calories', 'carbohydrates', 'sugars', 'fats', 'saturated_fats',
       'cholesterol', 'sodium', 'fiber', 'potassium', 'proteins'],
      dtype='object')
Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')
Index(['displayTime', 'Glucose Value'], dtype='object')


In [154]:
# Inspect unique food names in each dataset
print("Unique food names in Menu Data:")
print(menu_df['food_name'].unique())

print("\nUnique food names in Diabetic Food Data:")
print(diabetic_food_df['food_name'].unique())

print("\nUnique food names in Glycemic Index Data:")
print(gi_df['food_name'].unique())

Unique food names in Menu Data:
["Egg BLT McMuffin with Shredded Lettuce (McDonald's in Walmart)"
 'Cheeseburger' 'Hamburger' 'Honey' 'Hotcakes' 'McChicken' 'McCrispy'
 'McDouble' 'Americano, Large' 'Americano, Medium' 'Americano, Small'
 'Apple Slices' 'Big Breakfast' 'Big Mac' 'Cappuccino, Large'
 'Cappuccino, Medium' 'Cappuccino, Small' 'Dasani Water' 'Deluxe McCrispy'
 'Double Cheeseburger' 'Bacon King' 'Rodeo Burger' 'Whopper'
 'Bacon Cheeseburger' 'Bacon Melt' 'Big Fish' 'Chicken Jr.'
 'Chocolate Shake' 'Classic Melt' 'Double Whopper' 'Impossible Whopper'
 'Large Sprite' 'MOTTS Applesauce' 'Medium Sprite' 'Pancake Platter'
 'Philly Melt' 'Rodeo Cheeseburger'
 'Taco Shells, Bell Essentials, Crunchy' 'Crunchy Taco'
 'Nachos BellGrande - Beef' 'Nachos BellGrande - Chicken'
 'Nachos BellGrande - Steak' 'Crunchy Taco Supreme' 'Double Stacked Taco'
 'Soft Taco - Beef' 'Soft Taco - Chicken' 'Fries Bell Grande'
 'Nachos Bell Grande' 'Seasoning Mix, Original Taco'
 'Taco Tuesday - Crunchy

### Separating Grams from Food Names in Glycemic Index

In [156]:
# Extract weight (grams) from food_name
def extract_weight(name):
    match = re.search(r"\((.*?)\)", name)
    return match.group(1) if match else None

# Apply to glycemic index dataset
gi_df['serving_weight'] = gi_df['food_name'].apply(extract_weight)

# Remove the weight from food_name
gi_df['food_name'] = gi_df['food_name'].str.replace(r"\(.*?\)", "", regex=True).str.strip()

# Verify results
print(gi_df.head())

         food_name  glycemic_index serving_weight
0           Apples            40.0           120g
1      Apple juice            39.0           250g
2  Apricots, dried            32.0            60g
3          Bananas            47.0           120g
4   Fruit cocktail            55.0           120g


### Standardize Food Names Across Datasets

In [158]:
# Standardize food_name in all datasets
for df in [gi_df, menu_df, diabetic_food_df]:
    df['food_name'] = df['food_name'].str.lower().str.strip()

# Verify unique food names after standardization
print("Unique food names in Glycemic Index:", gi_df['food_name'].unique())
print("Unique food names in Menu Data:", menu_df['food_name'].unique())
print("Unique food names in Diabetic Food Data:", diabetic_food_df['food_name'].unique())

Unique food names in Glycemic Index: ['apples' 'apple juice' 'apricots, dried' 'bananas' 'fruit cocktail'
 'grapefruit' 'grapes' 'mangoes' 'oranges, raw'
 'peaches, canned in light syrup' 'pineapple' 'plums' 'strawberries'
 'carrot juice' 'carrots, raw' 'corn, sweet' 'lima beans, baby, frozen'
 'parsnips, peeled boiled' 'potato, white, boiled' 'tomato soup' 'barley'
 'basmati rice' 'bran cereal' 'brown rice, steamed'
 'bulgur wheat, whole, cooked' 'chickpeas' 'instant noodles'
 'instant oatmeal' 'mixed grain bread' 'oat bran bread' 'rye kernel bread'
 'rye flour bread, 50%\xa0 rye flour, 50% wheat flour'
 'water crackers, whole grain, sesame seeds' 'white rice, boiled'
 'skim milk' 'soy milk' 'black beans' 'butter beans' 'cashews'
 'kidney beans' 'kidney beans, canned' 'lentils, canned'
 'split peas, yellow, boiled' 'blueberry muffin' 'cake, pound'
 'corn chips' 'hummus' 'ice cream, full-fat, french vanilla'
 'ice cream, low-fat, vanilla, “light”' 'oatmeal cookies' 'snickers'
 'sponge 

### Align Columns Across Datasets

In [160]:
# # Ensure all datasets have the same columns
# all_columns = set(gi_df.columns) | set(menu_df.columns) | set(diabetic_food_df.columns)
# for df in [gi_df, menu_df, diabetic_food_df]:
#     for col in all_columns:
#         if col not in df.columns:
#             df[col] = pd.NA

# # Verify alignment
# print("Glycemic Index Columns:", gi_df.columns)
# print("Menu Data Columns:", menu_df.columns)
# print("Diabetic Food Data Columns:", diabetic_food_df.columns)

### Merge Data

In [162]:
# Merge Glycemic Index and Diabetic Food Data
food_gi_combined = pd.merge(diabetic_food_df, gi_df, on='food_name', how='outer')

# Add Menu Data
combined_food_data = pd.merge(food_gi_combined, menu_df, on='food_name', how='outer')

# Inspect the merged dataset
print("Combined Food Data:")
print(combined_food_data.head())

Combined Food Data:
  food_name                 category description          brand  \
0  broccoli  Diabetic-Friendly Foods    BROCCOLI    GREAT VALUE   
1  broccoli  Diabetic-Friendly Foods    BROCCOLI            NaN   
2  broccoli  Diabetic-Friendly Foods    BROCCOLI          MAMBO   
3  broccoli  Diabetic-Friendly Foods    BROCCOLI      EAT SMART   
4  broccoli  Diabetic-Friendly Foods    BROCCOLI  GOOD & GATHER   

                     food_category  calories_x  carbohydrates_x  fiber_x  \
0                Frozen Vegetables        24.0             4.71      3.5   
1  Pre-Packaged Fruit & Vegetables        34.0             5.41      2.0   
2                Frozen Vegetables        29.0             3.53      2.4   
3  Pre-Packaged Fruit & Vegetables        29.0             4.71      3.5   
4                Frozen Vegetables        24.0             4.71      3.5   

   sugars_x  fats_x  ...  calories_y  carbohydrates_y sugars_y fats_y  \
0       NaN    0.00  ...         3.5           

### Handle Duplicates

In [164]:
# Check for duplicates
duplicates = combined_food_data[combined_food_data.duplicated(subset='food_name', keep=False)]
print("Duplicate food names:")
print(duplicates)

Duplicate food names:
         food_name                 category description          brand  \
0         broccoli  Diabetic-Friendly Foods    BROCCOLI    GREAT VALUE   
1         broccoli  Diabetic-Friendly Foods    BROCCOLI            NaN   
2         broccoli  Diabetic-Friendly Foods    BROCCOLI          MAMBO   
3         broccoli  Diabetic-Friendly Foods    BROCCOLI      EAT SMART   
4         broccoli  Diabetic-Friendly Foods    BROCCOLI  GOOD & GATHER   
...            ...                      ...         ...            ...   
920          fries                      NaN         NaN            NaN   
971    plain wings                      NaN         NaN            NaN   
972    plain wings                      NaN         NaN            NaN   
1013  clam chowder                      NaN         NaN            NaN   
1014  clam chowder                      NaN         NaN            NaN   

                        food_category  calories_x  carbohydrates_x  fiber_x  \
0         

### Aggregate Numeric Columns

In [166]:
# Defining aggregation rules
numeric_aggregation = {
    'calories_x': 'mean',
    'carbohydrates_x': 'mean',
    'fiber_x': 'mean',
    'fats_x': 'mean',
    'glycemic_index': 'mean',
}
non_numeric_aggregation = {col: 'first' for col in combined_food_data.columns if col not in numeric_aggregation and col != 'food_name'}
aggregation_rules = {**numeric_aggregation, **non_numeric_aggregation}

# Aggregate data
aggregated_data = combined_food_data.groupby('food_name').agg(aggregation_rules).reset_index()

# Inspect aggregated data
print("Aggregated DataFrame with All Columns:")
print(aggregated_data.head())

Aggregated DataFrame with All Columns:
                          food_name  calories_x  carbohydrates_x  fiber_x  \
0                    #1 whataburger         NaN              NaN      NaN   
1        #2 double meat whataburger         NaN              NaN      NaN   
2        #3 triple meat whataburger         NaN              NaN      NaN   
3  #4 jalapeno & cheese whataburger         NaN              NaN      NaN   
4     #5 bacon & cheese whataburger         NaN              NaN      NaN   

   fats_x  glycemic_index category description brand food_category  ...  \
0     NaN             NaN     None        None  None          None  ...   
1     NaN             NaN     None        None  None          None  ...   
2     NaN             NaN     None        None  None          None  ...   
3     NaN             NaN     None        None  None          None  ...   
4     NaN             NaN     None        None  None          None  ...   

   calories_y  carbohydrates_y sugars_y fats_y 

### Normalize Nutritional Values

In [168]:
# Calculate per-100g values where serving_weight is available
aggregated_data['serving_weight'] = pd.to_numeric(aggregated_data['serving_weight'], errors='coerce')
valid_weight = aggregated_data['serving_weight'] > 0

aggregated_data.loc[valid_weight, 'calories_per_100g'] = (
    aggregated_data.loc[valid_weight, 'calories_x'] / aggregated_data.loc[valid_weight, 'serving_weight'] * 100
)
aggregated_data.loc[valid_weight, 'carbs_per_100g'] = (
    aggregated_data.loc[valid_weight, 'carbohydrates_x'] / aggregated_data.loc[valid_weight, 'serving_weight'] * 100
)
aggregated_data.loc[valid_weight, 'fats_per_100g'] = (
    aggregated_data.loc[valid_weight, 'fats_x'] / aggregated_data.loc[valid_weight, 'serving_weight'] * 100
)

Categorize Glycemic Index

In [170]:
# Categorize based on glycemic index
def categorize_gi(value):
    if pd.isna(value):
        return 'Unknown'
    elif value <= 55:
        return 'Low'
    elif value <= 70:
        return 'Medium'
    else:
        return 'High'

aggregated_data['gi_category'] = aggregated_data['glycemic_index'].apply(categorize_gi)


### Final Review

In [172]:
# Review final dataset
print("Final Dataset Shape:", aggregated_data.shape)
print(aggregated_data.head())

Final Dataset Shape: (725, 30)
                          food_name  calories_x  carbohydrates_x  fiber_x  \
0                    #1 whataburger         NaN              NaN      NaN   
1        #2 double meat whataburger         NaN              NaN      NaN   
2        #3 triple meat whataburger         NaN              NaN      NaN   
3  #4 jalapeno & cheese whataburger         NaN              NaN      NaN   
4     #5 bacon & cheese whataburger         NaN              NaN      NaN   

   fats_x  glycemic_index category description brand food_category  ...  \
0     NaN             NaN     None        None  None          None  ...   
1     NaN             NaN     None        None  None          None  ...   
2     NaN             NaN     None        None  None          None  ...   
3     NaN             NaN     None        None  None          None  ...   
4     NaN             NaN     None        None  None          None  ...   

   saturated_fats  cholesterol   sodium fiber_y  potass

### Export Merged data to CSV

In [174]:
# Export the final aggregated data to a CSV file
output_path = './data/combined_food_data.csv'
aggregated_data.to_csv(output_path, index=False)

print(f"Final combined data exported to: {output_path}")

Final combined data exported to: ./data/combined_food_data.csv


## Merging Patient Data

In [176]:
# Patient-Based Keys have standardized formatting
patient_df['Age'] = patient_df['Age'].astype(int)  # integer format
patient_df['Sex'] = patient_df['Sex'].astype(int)  # Standardize 'Sex' column

In [177]:
patient_df.columns
patient_df.shape

(33568, 22)

In [178]:
dexcom_cgm_df.columns
dexcom_cgm_df.shape

(17118, 2)

In [179]:
# Calculate the necessary repetitions of patient data to join with glucose data
num_repeats = len(dexcom_cgm_df) // len(patient_df) + 1
expanded_patient_df = pd.concat([patient_df] * num_repeats, ignore_index=True)
expanded_patient_df = expanded_patient_df[:len(dexcom_cgm_df)]  # Truncate to match glucose data length

# Combine patient data with Dexcom glucose data (should result in 17118 patient records)
simulated_df = pd.concat([expanded_patient_df.reset_index(drop=True), dexcom_cgm_df.reset_index(drop=True)], axis=1)

simulated_df['Time Checked'] = simulated_df['displayTime']
patient = simulated_df.drop(columns=['displayTime'])

# Display the simulated dataframe
patient.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Glucose Value,Time Checked
0,2.0,1,0.0,1,28.0,0.0,0.0,1.0,0,1,...,2.0,0.0,0.0,0.0,0,11,4.0,3.0,117,2024-11-05T14:18:32
1,2.0,1,1.0,1,33.0,0.0,0.0,0.0,1,1,...,2.0,10.0,0.0,0.0,0,9,4.0,7.0,115,2024-11-05T14:13:32
2,2.0,0,1.0,1,29.0,0.0,1.0,1.0,1,1,...,5.0,0.0,30.0,1.0,1,12,3.0,4.0,114,2024-11-05T14:08:32
3,2.0,0,0.0,1,24.0,0.0,0.0,1.0,0,0,...,4.0,0.0,0.0,0.0,1,12,6.0,7.0,115,2024-11-05T14:03:33
4,2.0,0,0.0,1,33.0,1.0,0.0,0.0,1,0,...,4.0,0.0,0.0,0.0,1,6,5.0,2.0,115,2024-11-05T13:58:32


In [180]:
# Validate record count
patient.shape

(17118, 24)

In [181]:
# Export the final aggregated data to a CSV file
output_path = './data/patient_records.csv'
patient.to_csv(output_path, index=False)

print(f"Final combined data exported to: {output_path}")

Final combined data exported to: ./data/patient_records.csv
